<a href="https://colab.research.google.com/github/eylulpelinkilic/Clash-Royale-DSA210-Project/blob/main/FINAL_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df = pd.read_excel("matches_50_players.xlsx")

In [ ]:
print("Unique values in 'win' column:", df['win'].unique())

Unique values in 'win' column: [1 0]


In [ ]:
nan_summary = df.isnull().sum()
print("Columns with missing values:\n", nan_summary[nan_summary > 0])

Columns with missing values:
 Series([], dtype: int64)


In [ ]:
# 1. Necessary imports
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter

# 2. Load data
df = pd.read_excel("matches_50_players.xlsx")
card_features_df = pd.read_excel("card_types_full_multi_labeled.xlsx")

# 3. Parse card_type correctly
def safe_list(x):
    if isinstance(x, list):
        return x
    try:
        return ast.literal_eval(x)
    except:
        return []

card_features_df['card_type'] = card_features_df['card_type'].apply(safe_list)

# 4. Parse decks (team and opponent) correctly
df['team_deck'] = df['team_deck'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['opponent_deck'] = df['opponent_deck'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# 5. One-Hot Encoding for Cards
mlb_team = MultiLabelBinarizer()
team_ohe = pd.DataFrame(mlb_team.fit_transform(df['team_deck']), columns=[f"team_has_{card}" for card in mlb_team.classes_])

mlb_opponent = MultiLabelBinarizer(classes=mlb_team.classes_)
opponent_ohe = pd.DataFrame(mlb_opponent.fit_transform(df['opponent_deck']), columns=[f"opponent_has_{card}" for card in mlb_team.classes_])

team_ohe.index = df.index
opponent_ohe.index = df.index

df = pd.concat([df, team_ohe, opponent_ohe], axis=1)

# 6. Create dictionaries for card properties
card_to_types = dict(zip(card_features_df['card_name'], card_features_df['card_type']))
card_to_elixir = dict(zip(card_features_df['card_name'], card_features_df['elixir']))

# 7. Extract all unique types
all_types = set()
for types in card_features_df['card_type']:
    all_types.update(types)

# 8. Feature extraction function
def extract_deck_features(deck):
    type_counter = Counter()
    elixir_total = 0
    n_cards = len(deck)

    for card in deck:
        types = card_to_types.get(card, [])
        for t in types:
            type_counter[t] += 1
        elixir_total += card_to_elixir.get(card, 0)

    features = {f"n_{t}": type_counter[t] for t in all_types}
    features["avg_elixir"] = elixir_total / n_cards if n_cards > 0 else 0
    return features

# 9. Apply feature extraction
team_features = df['team_deck'].apply(extract_deck_features).apply(pd.Series)
opponent_features = df['opponent_deck'].apply(extract_deck_features).apply(pd.Series)

# 10. Remove duplicate columns
team_features = team_features.loc[:, ~team_features.columns.duplicated()]
opponent_features = opponent_features.loc[:, ~opponent_features.columns.duplicated()]

# 11. Prefix columns
team_features = team_features.add_prefix('team_')
opponent_features = opponent_features.add_prefix('opponent_')

# 12. Merge
df = pd.concat([df, team_features, opponent_features], axis=1)

# 13. Calculate type differences
valid_types = []
for t in all_types:
    team_col = f'team_n_{t}'
    opponent_col = f'opponent_n_{t}'
    if team_col in df.columns and opponent_col in df.columns:
        valid_types.append(t)

for t in valid_types:
    team_col = f'team_n_{t}'
    opponent_col = f'opponent_n_{t}'
    df[f'diff_n_{t}'] = df[team_col] - df[opponent_col]

# Drop duplicate elixir columns (keep only the first occurrence)
cols_to_drop = [col for col in df.columns if (col.startswith('team_avg_elixir') or col.startswith('opponent_avg_elixir')) and ('.' in col)]

df = df.drop(columns=cols_to_drop)

df.to_excel("without_diff_avg_elixir.xlsx", index=False)
from google.colab import files
files.download("without_diff_avg_elixir.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = pd.read_excel("without_diff_avg_elixir.xlsx")
df['diff_avg_elixir'] = df['team_avg_elixir'] - df['opponent_avg_elixir']
df.to_excel("matches_50_players_prepared.xlsx", index=False)
from google.colab import files
files.download("matches_50_players_prepared.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df.to_excel("matches_50_players_prepared.xlsx", index=False)
from google.colab import files
files.download("matches_50_players_prepared.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>